Detección de caras con webcam con detectores de OpenCV

In [5]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

# Models load and setup
# VJ
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# DNN
dnn_model = "deploy.prototxt.txt"
dnn_weights = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(dnn_model, dnn_weights)
# MTCNN
mtcnn_det = MTCNN()

# Webcam connection
video_capture = cv2.VideoCapture(0)

# Detector index
det = 1
while True:
    ret, frame = video_capture.read()    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Depending on the active detector
    if det == 0:
        # VJ
        faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Draw bounding boxes
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    elif det == 1:
        # DNN
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104.0, 177.0, 123.0], False, False)
        net.setInput(blob)
        detections = net.forward()

        # Draw bounding boxes
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x2, y2) = box.astype("int")
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
    else: #MTCNN
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mtcnn_det.detect_faces(rgb_frame)

        # Draw bounding boxes
        for result in results:
            x, y, width, height = result['box']
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)
    
            # Obtener los puntos clave
            keypoints = result['keypoints']
            
            # Extraer coordenadas de los ojos
            left_eye = keypoints['left_eye']
            right_eye = keypoints['right_eye']

            # Dibujar los ojos (círculos verdes en los ojos)
            cv2.circle(frame, left_eye, 5, (0, 255, 0), 2)
            cv2.circle(frame, right_eye, 5, (0, 255, 0), 2)

            # (Opcional) Dibujar otros puntos clave como nariz y boca
            cv2.circle(frame, keypoints['nose'], 5, (255, 0, 0), 2)
            cv2.circle(frame, keypoints['mouth_left'], 5, (0, 0, 255), 2)
            cv2.circle(frame, keypoints['mouth_right'], 5, (0, 0, 255), 2)
    # Shgow results
    cv2.imshow('Face Detection', frame)
    
    # 'd' to change detector, Esc to finish
    key = cv2.waitKey(1) & 0xFF
    if key == ord('d'):
        det += 1
        if det > 2:
            det = 0
    elif key == 27:
        break

# Close windoews and release camera
video_capture.release()
cv2.destroyAllWindows()




KeyboardInterrupt: 

In [6]:
import cv2
from mtcnn import MTCNN

# Inicializar el detector MTCNN
mtcnn_det = MTCNN()

# Capturar video desde la cámara
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a RGB (MTCNN espera imágenes en RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detectar caras y puntos clave
    results = mtcnn_det.detect_faces(rgb_frame)

    # Verificar si hay resultados
    if results:
        # Dibujar recuadros y puntos clave (ojos, nariz, boca)
        for result in results:
            # Dibujar la caja delimitadora de la cara
            x, y, width, height = result['box']
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)

            # Obtener los puntos clave
            keypoints = result.get('keypoints', {})
            
            # Verificar si hay puntos clave (ojos, nariz, boca)
            if keypoints:
                left_eye = keypoints.get('left_eye')
                right_eye = keypoints.get('right_eye')
                nose = keypoints.get('nose')
                mouth_left = keypoints.get('mouth_left')
                mouth_right = keypoints.get('mouth_right')

                # Comprobar si los ojos fueron detectados
                if left_eye and right_eye:
                    # Dibujar los ojos (círculos verdes en los ojos)
                    cv2.circle(frame, left_eye, 5, (0, 255, 0), 2)
                    cv2.circle(frame, right_eye, 5, (0, 255, 0), 2)

                # Dibujar otros puntos clave si están disponibles
                if nose:
                    cv2.circle(frame, nose, 5, (255, 0, 0), 2)
                if mouth_left:
                    cv2.circle(frame, mouth_left, 5, (0, 0, 255), 2)
                if mouth_right:
                    cv2.circle(frame, mouth_right, 5, (0, 0, 255), 2)

    else:
        print("No se detectaron caras.")

    # Mostrar el frame con los recuadros y puntos clave
    cv2.imshow("Detección de Rostros", frame)

    # Salir si se presiona 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


Left eye: [np.int64(301), np.int64(276)], Right eye: [np.int64(371), np.int64(283)]
Left eye: [np.int64(304), np.int64(275)], Right eye: [np.int64(369), np.int64(281)]
Left eye: [np.int64(304), np.int64(276)], Right eye: [np.int64(370), np.int64(284)]
Left eye: [np.int64(306), np.int64(276)], Right eye: [np.int64(372), np.int64(285)]
Left eye: [np.int64(305), np.int64(276)], Right eye: [np.int64(375), np.int64(280)]
Left eye: [np.int64(229), np.int64(231)], Right eye: [np.int64(236), np.int64(232)]
Left eye: [np.int64(305), np.int64(276)], Right eye: [np.int64(371), np.int64(283)]
Left eye: [np.int64(305), np.int64(273)], Right eye: [np.int64(371), np.int64(278)]
Left eye: [np.int64(307), np.int64(269)], Right eye: [np.int64(372), np.int64(272)]
Left eye: [np.int64(307), np.int64(266)], Right eye: [np.int64(371), np.int64(270)]
Left eye: [np.int64(307), np.int64(266)], Right eye: [np.int64(369), np.int64(269)]
Left eye: [np.int64(306), np.int64(263)], Right eye: [np.int64(370), np.int6

In [2]:
import cv2
from mtcnn import MTCNN
import numpy as np

# Inicializar el detector MTCNN
mtcnn_det = MTCNN()

# Cargar la imagen de las gafas de sol con fondo transparente
gafas = cv2.imread('gafas.png', cv2.IMREAD_UNCHANGED)  # Cargar como PNG con canal alfa (transparente)

# Función para superponer una imagen con transparencia
def superponer_imagen(bg, fg, x, y, w, h):
    # Redimensionar la imagen de las gafas al ancho y alto deseado
    fg = cv2.resize(fg, (w, h))

    # Obtener los canales de la imagen (RGBA)
    for i in range(3):  # Solo combinar los canales RGB
        bg[y:y+h, x:x+w, i] = fg[:, :, i] * (fg[:, :, 3] / 255.0) + bg[y:y+h, x:x+w, i] * (1 - fg[:, :, 3] / 255.0)

# Capturar video desde la cámara
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a RGB (MTCNN espera imágenes en RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detectar caras y puntos clave
    results = mtcnn_det.detect_faces(rgb_frame)

    # Verificar si hay resultados
    if results:
        for result in results:
            # Dibujar la caja delimitadora de la cara
            x, y, width, height = result['box']
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)

            # Obtener los puntos clave (ojos)
            keypoints = result.get('keypoints', {})
            if keypoints:
                left_eye = keypoints.get('left_eye')
                right_eye = keypoints.get('right_eye')

                if left_eye and right_eye:
                    # Calcular la distancia entre los ojos
                    eye_width = int(np.linalg.norm(np.array(left_eye) - np.array(right_eye)))

                    # Calcular las coordenadas y dimensiones para las gafas
                    gafas_width = eye_width * 2  # Ancho de las gafas
                    gafas_height = int(gafas_width * gafas.shape[0] / gafas.shape[1])  # Mantener la proporción de las gafas
                    gafas_x = left_eye[0] - eye_width // 2  # Centrar en el ojo izquierdo
                    gafas_y = left_eye[1] - gafas_height // 2  # Ajustar verticalmente

                    # Superponer las gafas
                    superponer_imagen(frame, gafas, gafas_x, gafas_y, gafas_width, gafas_height)

    # Mostrar el frame con los recuadros y gafas de sol
    cv2.imshow("Detección de Rostros con Gafas", frame)

    # Salir si se presiona 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
